In [3]:
import requests

headers = {
    'accept': '*/*',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'access-control-request-headers': 'cache,frontend',
    'access-control-request-method': 'GET',
    'origin': 'https://consulteonline.conlicitacao.com.br',
    'priority': 'u=1, i',
    'referer': 'https://consulteonline.conlicitacao.com.br/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36',
}

params = {
    'page': '2',
    'per_page': '50',
}

response = requests.options(
    'https://consultaonline.conlicitacao.com.br/boletim_web/public/boletins/134665066/biddings.json',
    params=params,
    headers=headers,
)

print(response)

<Response [200]>
